In [ ]:
from pymongo import MongoClient
from pprint import pprint

client = MongoClient()

### Exercise 1

In [ ]:
!mongoimport /d lab /c inventory ../datasets/inventory.json --maintainInsertionOrder

db = client.lab
col = db.inventory

In [ ]:
# Exercise 1-1
result = col.find(
    {"tags":{
        '$all': ['appliance', 'school', 'book']
        }
    })
pprint(list(result)) 

In [ ]:
# Exercise 1-2
result = col.find({}, {'tags': {'$slice': [1, 3]}})
pprint(list(result))

In [ ]:
# Exercise 1-3
result = col.find({'qty': {'$elemMatch': {'color': 'blue', 'size': '6'}}})
pprint(list(result))

### Exercise 2

In [ ]:
! mongoimport /d lab /c store ../datasets-master/store.json --maintainInsertionOrder

store = db.store
store.find_one()

In [ ]:
# Exercise 2-1
result = store.find({'items.name': "iPhone Xs"},
                   {'items': 1, '_id': 0 })
pprint(list(result))

In [ ]:
# Exercise 2-2
result = store.find({'items.price': {'$lt': 400}},
                   {'buyer.name': 1, '_id': 0})
pprint(list(result))

In [ ]:
# Exercise 2-3
result = store.find({
    'items': {'$elemMatch': {'name': "Apple Watch Series 4",
                            'attribute.carrier': {'$exists': False}}}
})
pprint(list(result))

### Exercise 3

In [ ]:
# Exercise 3-1
inventory = db.inventory
res1 = inventory.count_documents({})
res2 = inventory.find().limit(2)

print(res1)
pprint(list(res2))

In [ ]:
# Exercise 3-2
res = store.find({}, {'buyer.method': 1, '_id': 0}).skip(3).limit(2)
pprint(list(res))

### Exercise 4

In [ ]:
posts = client.lab.posts

# Exercise 4-1
posts.insert_one({
    'name': 'Lee',
    'content': "Hello I'm Lee",
    'tags': ['Hello', 'Greet']})

In [ ]:
posts.update_one({'name': 'Lee'}, {
    '$push': {'comments':
        {'$each' : [
            {"name" : "Kim", "content" : "Good posts.", "like" : 3},
            {"name" : "Choi", "content" : "How is it going?", "like" : 1},
            {"name" : "David", "content" : "Hi, I’m David.", "like" : 2},
            {"name" : "Kim", "content" : "Welcome!", "like" : 3}
            ]
        }
    }
})

In [ ]:
res = posts.find()
pprint(list(res))

In [ ]:
# Exercise 4-2
posts.update_one(
    {'name': 'Lee'},
    {'$set': {'comments.1.like': 0}})

posts.update_one(
    {'name': 'Lee', 'comments.name': 'Kim'},
    {'$set': {'comments.$.like': 5}})

res = posts.find()
pprint(list(res))

In [ ]:
# Exercise 4-3
posts.update_one(
    {'name': 'Lee'},
    {'$set': {'comments.$[var].like': 6}},
    array_filters = [{'var.like': {'$gte': 3}}])

res = posts.find()
pprint(list(res))

In [ ]:
# Exercise 4-4
posts.update_one(
    {'name': 'Lee'},
    {'$set': {'comments.$[].like': 1}})

res = posts.find()
pprint(list(res))

In [ ]:
# Exercise 4-5
posts.update_one(
    {'name': 'Lee'},
    {'$pop': {'comments': 1}})

posts.update_one(
    {'name': 'Lee'},
    {'$pull': {'comments': {'name': 'Choi'}}})

posts.update_one(
    {'name': 'Lee'},
    {'$unset': {'content': 1}})

res = posts.find()
pprint(list(res))

### Exercise 5

In [ ]:
!mongoimport /d lab /c grade ./dataset/grade.json
from pymongo import MongoClient
from pprint import pprint

client = MongoClient()

col = client.lab.grade

#### Exercise 5-1

In [ ]:
# Print lost document’s sid and Insert the quiz score (80) to the corresponding sid 
total_count = (col.count_documents({}) + 1) // 3

for sid in range(total_count):
    count = col.count_documents({'sid': sid})
    if count != 3:
        print(sid)
        col.insert_one({'sid': sid, 'type': 'quiz', 'score': 80})
        break

In [ ]:
## Print the three students that have the highest quiz score.
result = col.find({'type':'quiz'},{"_id":0}).sort([('score', -1)]).limit(3)
pprint(list(result))

#### Exercise 5-2

In [ ]:
raw_input = input('search >')
params = raw_input.split()

target_1, target_2 = None, None
try:
    target_1 = int(params[1])
except:
    split_result = params[1].split('-')
    target_1, target_2 = int(split_result[0]), int(split_result[1])

# query 부분
result_cursor = None
if target_2 is None:
    result_cursor = col.find({'type': params[0],
                              'score': target_1})
else:
    result_cursor = col.find({'type': params[0], 
                              'score': {
                                  '$gte': target_1, 
                                  '$lte': target_2}})

print('sid\t{}'.format(params[0]))
for item in result_cursor:
    print('{}\t{}'.format(item['sid'], item['score']))